In [5]:
import json
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

# ---------- LOAD TRAINED DATA ----------
with open("msme_synthetic_cases.json") as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Encode categorical columns (same as training)
dispute_encoder = LabelEncoder()
state_encoder = LabelEncoder()

df["dispute_type_enc"] = dispute_encoder.fit_transform(df["dispute_type"])
df["jurisdiction_enc"] = state_encoder.fit_transform(df["jurisdiction"])

FEATURES = [
    "claim_amount",
    "delay_days",
    "document_count",
    "document_completeness_score",
    "dispute_type_enc",
    "jurisdiction_enc"
]

X = df[FEATURES]
y = df["is_settlement"]

# ---------- TRAIN MODEL AGAIN (simple way) ----------
model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    eval_metric="logloss",
    random_state=42
)

model.fit(X, y)

# ---------- FAKE MSME CASE ----------
fake_case = {
    "claim_amount": 48000,             # ₹4.8 lakh
    "delay_days": 132,                  # delayed by 132 days
    "document_count": 3,                # invoice + PO + email
    "document_completeness_score": 0.75,
    "dispute_type": "invoice_non_payment",
    "jurisdiction": "Maharashtra"
}

# ---------- PREPROCESS FAKE CASE ----------
fake_df = pd.DataFrame([fake_case])

fake_df["dispute_type_enc"] = dispute_encoder.transform(fake_df["dispute_type"])
fake_df["jurisdiction_enc"] = state_encoder.transform(fake_df["jurisdiction"])

fake_X = fake_df[FEATURES]

# ---------- RUN MODEL ----------
probability = model.predict_proba(fake_X)[0][1]

# ---------- SIMPLE INTERPRETATION ----------
if probability > 0.6:
    priority = "High settlement potential"
elif probability > 0.3:
    priority = "Medium settlement potential"
else:
    priority = "Low settlement potential"

# ---------- OUTPUT ----------
print("\n📄 MSME CASE ANALYSIS")
print("---------------------")
print(f"Settlement Probability : {round(probability * 100, 2)}%")
print(f"Priority Level         : {priority}")

print("\n🧠 AI Reasoning (simplified):")
print("- Delay duration:", fake_case["delay_days"], "days")
print("- Document completeness:", fake_case["document_completeness_score"])
print("- Claim amount: ₹", fake_case["claim_amount"])



📄 MSME CASE ANALYSIS
---------------------
Settlement Probability : 50.99%
Priority Level         : Medium settlement potential

🧠 AI Reasoning (simplified):
- Delay duration: 132 days
- Document completeness: 0.75
- Claim amount: ₹ 48000


In [6]:
pip install streamlit pandas scikit-learn xgboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 772.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 44.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 42.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 816.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 1.3 MB/s eta 0:00:00eta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.3.1
    Uninstalling cachetools-5.3.1:
      Successfully uninstalled cachetools-5.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-auth 2.23.3 req